# CIR-2 Baseline Imputation Methods

In [1]:
import pandas as pd
import numpy as np
import os
import io
import logging
import time
import datetime
import random
import matplotlib.pyplot as plt
import sys

from tqdm import tqdm

from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import BayesianRidge

In [2]:
# Initial logger setup
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Global variable to hold the active file handler
current_file_handler = None

# Create the stream handler
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

def switch_log_file(filename):
    global current_file_handler

    # If a file handler already exists, remove and close it
    if current_file_handler:
        logger.removeHandler(current_file_handler)
        current_file_handler.close()

    # Create a new file handler
    current_file_handler = logging.FileHandler(filename)
    current_file_handler.setFormatter(formatter)
    logger.addHandler(current_file_handler)

    logger.info(f"Switched logging to {filename}")

In [3]:
# Build log file
switch_log_file('logs/CIR-2.log')
logger.info("This is being logged to CIR-2.log")

2025-04-28 23:32:18,472 - INFO - Switched logging to logs/CIR-2.log
2025-04-28 23:32:18,475 - INFO - This is being logged to CIR-2.log


In [4]:
# Load datasets
data_path = "../04_ANN/CSV/exports/split_set/without_multiple_rows"
all_files = os.listdir(data_path)

logging.info("+++++++++++++++++CIR-2+++++++++++++++++++++++++")
logging.info("Start Loading Dataframes.")

# Load CSVs into a dictionary of dataframes
dataframes = {}
for file in all_files:
    if file.endswith(".csv"):
        var_name = file.replace(".csv", "").replace("-", "_")
        logging.info(f"Loading... -> {file}")
        dataframes[var_name] = pd.read_csv(os.path.join(data_path, file)).astype('float32')

# Log loaded datasets
for var_name, df in dataframes.items():
    globals()[var_name] = df
    logging.info(f"{var_name} loaded successfully with shape {df.shape}")
logging.info("Load Complete.")
logging.info("++++++++++++++++++++++++++++++++++++++++++")

2025-04-28 23:32:18,487 - INFO - +++++++++++++++++CIR-2+++++++++++++++++++++++++
2025-04-28 23:32:18,489 - INFO - Start Loading Dataframes.
2025-04-28 23:32:18,490 - INFO - Loading... -> o1_X_external.csv
2025-04-28 23:32:25,322 - INFO - Loading... -> o1_X_test.csv
2025-04-28 23:32:25,831 - INFO - Loading... -> o1_X_train.csv
2025-04-28 23:32:29,921 - INFO - Loading... -> o1_X_validate.csv
2025-04-28 23:32:30,468 - INFO - Loading... -> o1_y_external_los.csv
2025-04-28 23:32:30,514 - INFO - Loading... -> o1_y_external_mortality.csv
2025-04-28 23:32:30,549 - INFO - Loading... -> o1_y_test_los.csv
2025-04-28 23:32:30,561 - INFO - Loading... -> o1_y_test_mortality.csv
2025-04-28 23:32:30,566 - INFO - Loading... -> o1_y_train_los.csv
2025-04-28 23:32:30,605 - INFO - Loading... -> o1_y_train_mortality.csv
2025-04-28 23:32:30,624 - INFO - Loading... -> o1_y_validate_los.csv
2025-04-28 23:32:30,634 - INFO - Loading... -> o1_y_validate_mortality.csv
2025-04-28 23:32:30,639 - INFO - Loading... -

# CIR-23 Implement Iterative Imputer

In [5]:
# Tee class to duplicate stdout
class Tee:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()

def impute_with_iterative(input_df, method, output_path, n_iter, log_verbose_file_path=None):
    logging.info(f"Starting Iterative Imputer with method={method} on input DataFrame of shape {input_df.shape}.")

    data_copy = input_df.copy()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Estimator selection
    if method == "ExtraTrees":
        estimator = ExtraTreesRegressor(n_estimators=10, random_state=0, n_jobs=-1)
    elif method == "HistGradientBoosting":
        estimator = HistGradientBoostingRegressor(random_state=0)
    else:
        raise ValueError(f"Unsupported method: {method}. Use 'ExtraTrees', or 'HistGradientBoosting'.")

    # IterativeImputer
    imputer = IterativeImputer(
        estimator=estimator,
        max_iter=n_iter,
        random_state=0,
        verbose=2,
        sample_posterior=False
    )

    start_time = time.time()

    # Handle logging duplication
    if log_verbose_file_path is not None:
        log_dir = os.path.dirname(log_verbose_file_path)
        os.makedirs(log_dir, exist_ok=True)

        original_stdout = sys.stdout  # Save original stdout
        with open(log_verbose_file_path, "w") as log_file:
            sys.stdout = Tee(sys.__stdout__, log_file)

            try:
                # Fit and transform
                imputed_array = imputer.fit_transform(data_copy)
            finally:
                sys.stdout = original_stdout  # Restore stdout
    else:
        # If no logging redirection
        imputed_array = imputer.fit_transform(data_copy)

    end_time = time.time()
    runtime = end_time - start_time

    # Save imputed data
    imputed_df = pd.DataFrame(imputed_array, columns=data_copy.columns)
    imputed_df.to_csv(output_path, index=False)

    # Logging
    logging.info(f"Imputation completed in {runtime:.2f} seconds.")
    nan_count = np.isnan(imputed_df.values).sum()
    logging.info(f"Number of NaNs after imputation: {nan_count}")
    logging.info(f"Imputed dataset saved at {output_path}")
    logging.info(f"Basic statistics after imputation:\n{imputed_df.describe()}")

    # Save full describe output to a separate CSV file
    # The logging.info could not represent all the statistics
    describe_output_path = output_path.replace(".csv", "_describe.csv")
    imputed_df.describe().to_csv(describe_output_path)
    logging.info(f"Basic statistics saved at {describe_output_path}")

    return imputed_df

In [6]:
# Create log file
switch_log_file('logs/CIR-23_ExtraTrees.log')
logger.info("This is being logged to CIR-23_ExtraTrees.log")

# ExtraTrees estimator
impute_with_iterative(
    input_df=o4_X_train,
    output_path="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o4_X_train_Iterative_ExtraTrees.csv",
    method="ExtraTrees",
    n_iter=3,
    log_verbose_file_path="logs/CIR-23_ExtraTrees.log"
)

2025-04-28 23:33:00,516 - INFO - Switched logging to logs/CIR-23_ExtraTrees.log
2025-04-28 23:33:00,517 - INFO - Starting Iterative Imputer with method=ExtraTrees on input DataFrame of shape (30624, 345).
C:\Users\Dimopoulos\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
2025-04-29 00:36:28,031 - INFO - Imputation completed in 3799.61 seconds.
2025-04-29 00:36:28,051 - INFO - Number of NaNs after imputation: 0
2025-04-29 00:36:28,053 - INFO - Imputed dataset saved at CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o4_X_train_Iterative_ExtraTrees.csv
2025-04-29 00:36:28,946 - INFO - Basic statistics after imputation:
                age  Alanine_Aminotransferase_(ALT)_(Max)  \
count  30624.000000                          30624.000000   
mean      69.297020                             75.495621   
std       14.930233                            221.375717   
min       18.000000

,age,Alanine_Aminotransferase_(ALT)_(Max),Alanine_Aminotransferase_(ALT)_(Mean),Alanine_Aminotransferase_(ALT)_(Median),Alanine_Aminotransferase_(ALT)_(Min),Albumin_(Max),Albumin_(Mean),Albumin_(Median),Albumin_(Min),Alkaline_Phosphatase_(Max),...,race_PATIENT DECLINED TO ANSWER,race_PORTUGUESE,race_SOUTH AMERICAN,race_UNABLE TO OBTAIN,race_UNKNOWN,race_WHITE,race_WHITE - BRAZILIAN,race_WHITE - EASTERN EUROPEAN,race_WHITE - OTHER EUROPEAN,race_WHITE - RUSSIAN
0,51.0,46.0,46.0,46.0,46.0,2.90,2.900,2.900,2.90,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,51.0,46.0,46.0,46.0,46.0,2.90,2.900,2.900,2.90,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,51.0,46.0,46.0,46.0,46.0,2.90,2.900,2.900,2.90,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51.0,46.0,46.0,46.0,46.0,2.90,2.900,2.900,2.90,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,51.0,46.0,46.0,46.0,46.0,2.90,2.900,2.900,2.90,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30619,57.0,63.0,63.0,63.0,63.0,3.64,3.475,3.520,3.36,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30620,57.0,63.0,63.0,63.0,63.0,3.62,3.490,3.530,3.41,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30621,57.0,63.0,63.0,63.0,63.0,3.64,3.475,3.520,3.36,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30622,57.0,63.0,63.0,63.0,63.0,3.64,3.475,3.525,3.34,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Create log file
switch_log_file('logs/CIR-23_HistGradientBoosting.log')
logger.info("This is being logged to CIR-23_HistGradientBoosting.log")

# HistGradientBoosting estimator
impute_with_iterative(
    input_df=o4_X_train,
    output_path="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o4_X_train_Iterative_HistGradientBoosting.csv",
    method="HistGradientBoosting",
    n_iter=20,
    log_verbose_file_path="logs/CIR-23_HistGradientBoosting.log"
)

2025-04-29 00:36:30,147 - INFO - Switched logging to logs/CIR-23_HistGradientBoosting.log
2025-04-29 00:36:30,148 - INFO - This is being logged to CIR-23_HistGradientBoosting.log
2025-04-29 00:36:30,150 - INFO - Starting Iterative Imputer with method=HistGradientBoosting on input DataFrame of shape (30624, 345).
C:\Users\Dimopoulos\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
2025-04-29 12:22:43,347 - INFO - Imputation completed in 42364.22 seconds.
2025-04-29 12:22:43,369 - INFO - Number of NaNs after imputation: 0
2025-04-29 12:22:43,374 - INFO - Imputed dataset saved at CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o4_X_train_Iterative_HistGradientBoosting.csv
2025-04-29 12:22:44,580 - INFO - Basic statistics after imputation:
                age  Alanine_Aminotransferase_(ALT)_(Max)  \
count  30624.000000                          30624.000000   
mean      69.297020  

,age,Alanine_Aminotransferase_(ALT)_(Max),Alanine_Aminotransferase_(ALT)_(Mean),Alanine_Aminotransferase_(ALT)_(Median),Alanine_Aminotransferase_(ALT)_(Min),Albumin_(Max),Albumin_(Mean),Albumin_(Median),Albumin_(Min),Alkaline_Phosphatase_(Max),...,race_PATIENT DECLINED TO ANSWER,race_PORTUGUESE,race_SOUTH AMERICAN,race_UNABLE TO OBTAIN,race_UNKNOWN,race_WHITE,race_WHITE - BRAZILIAN,race_WHITE - EASTERN EUROPEAN,race_WHITE - OTHER EUROPEAN,race_WHITE - RUSSIAN
0,51.0,46.0,46.0,46.0,46.0,2.900000,2.900000,2.900000,2.900000,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,51.0,46.0,46.0,46.0,46.0,2.900000,2.900000,2.900000,2.900000,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,51.0,46.0,46.0,46.0,46.0,2.900000,2.900000,2.900000,2.900000,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51.0,46.0,46.0,46.0,46.0,2.900000,2.900000,2.900000,2.900000,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,51.0,46.0,46.0,46.0,46.0,2.900000,2.900000,2.900000,2.900000,113.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30619,57.0,63.0,63.0,63.0,63.0,3.489160,3.451990,3.450475,3.441440,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30620,57.0,63.0,63.0,63.0,63.0,3.487714,3.451509,3.450464,3.441509,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30621,57.0,63.0,63.0,63.0,63.0,3.487733,3.451496,3.450602,3.444929,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30622,57.0,63.0,63.0,63.0,63.0,3.487801,3.452002,3.450602,3.440506,106.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# Custom Iterative Imputer
## I should see it again

In [ ]:
import os
import sys
import time
import logging
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import clone

def custom_iterative_imputer(input_df, method, output_folder, n_iter=5, random_state=0):
    """
    Custom Iterative Imputer that saves outputs after each iteration,
    calculates Change and Scaled Tolerance.

    Parameters:
    - input_df: pd.DataFrame to impute.
    - method: "ExtraTrees", "BayesianRidge", or "HistGradientBoosting".
    - output_folder: Folder to save intermediate outputs.
    - n_iter: Number of maximum iterations.
    """
    logging.info(f"Starting Custom Iterative Imputer with method={method} on input DataFrame of shape {input_df.shape}.")
    os.makedirs(output_folder, exist_ok=True)
    
    data_copy = input_df.copy()

    # Choose estimator
    if method == "ExtraTrees":
        base_estimator = ExtraTreesRegressor(n_estimators=10, random_state=random_state, n_jobs=-1)
    elif method == "BayesianRidge":
        base_estimator = BayesianRidge()
    elif method == "HistGradientBoosting":
        base_estimator = HistGradientBoostingRegressor(random_state=random_state)
    else:
        raise ValueError(f"Unsupported method: {method}")

    # Initial mean fill to avoid full NaN columns
    simple_imputer = SimpleImputer(strategy="mean")
    X_filled = pd.DataFrame(simple_imputer.fit_transform(data_copy), columns=data_copy.columns).astype(np.float32)

    # Save initial state for comparison
    X_prev = X_filled.copy()

    start_time = time.time()

    for round_idx in range(1, n_iter + 1):
        logging.info(f"Starting iteration {round_idx}/{n_iter}")

        for feature_idx, feature in enumerate(X_filled.columns):
            if data_copy[feature].isnull().sum() == 0:
                continue  # No missing values to predict for this feature

            # Split data
            known_mask = ~data_copy[feature].isnull()
            unknown_mask = data_copy[feature].isnull()

            if known_mask.sum() == 0:
                logging.warning(f"Skipping feature {feature} entirely missing.")
                continue

            X_known = X_filled.loc[known_mask].drop(columns=[feature])
            y_known = X_filled.loc[known_mask, feature]
            X_unknown = X_filled.loc[unknown_mask].drop(columns=[feature])

            if X_unknown.shape[0] == 0:
                continue

            # Clone the base estimator each time to avoid contamination
            estimator = clone(base_estimator)
            estimator.fit(X_known, y_known)
            y_pred = estimator.predict(X_unknown)

            # Fix dtype to avoid FutureWarning
            X_filled.loc[unknown_mask, feature] = y_pred.astype(np.float32)

        # Calculate Change and Scaled Tolerance
        change = np.abs(X_filled.values - X_prev.values).sum()
        scaled_tolerance = np.mean(np.abs(X_prev.values)) * 1e-3

        logging.info(f"Ending iteration {round_idx}/{n_iter}")
        logging.info(f"Change: {change:.6f}, Scaled Tolerance: {scaled_tolerance:.6f}")

        # Save after each round
        round_csv = os.path.join(output_folder, f"imputed_round_{round_idx}.csv")
        round_desc = os.path.join(output_folder, f"imputed_round_{round_idx}_describe.csv")

        X_filled.to_csv(round_csv, index=False)
        X_filled.describe().to_csv(round_desc)

        logging.info(f"Iteration {round_idx} completed and saved to {round_csv}")

        # Update X_prev for next round
        X_prev = X_filled.copy()

    end_time = time.time()
    runtime = end_time - start_time

    logging.info(f"Custom Iterative Imputer completed in {runtime:.2f} seconds.")

    return X_filled

In [ ]:
# Switch to new log file if needed
switch_log_file('logs/CIR-24_Custom_Iterative.log')

# Call custom iterative imputer
final_imputed_df = custom_iterative_imputer(
    input_df=small_data,
    method="ExtraTrees",  # or "HistGradientBoosting", "BayesianRidge"
    output_folder="CSV/exports/CRI-02/o1_impute_baselines/01_custom_iterative/",
    n_iter=3  # for example
)

# Plot

In [ ]:
def plot_before_after_distributions(input_df_before, input_df_after, output_folder, sample_features=None):
    """
    Plot before and after distributions for selected features and save the figures.
    
    If sample_features is None, it plots for all features.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    features = list(input_df_before.columns)

    # If sample_features is None ➔ use all features
    if sample_features is None:
        sampled_features = features
    else:
        sampled_features = random.sample(features, min(sample_features, len(features)))
    
    logging.info(f"Plotting distributions for features: {sampled_features}")

    for feature in sampled_features:
        plt.figure(figsize=(10,5))
        
        plt.hist(input_df_before[feature].dropna(), bins=50, alpha=0.5, label='Before Imputation')
        plt.hist(input_df_after[feature].dropna(), bins=50, alpha=0.5, label='After Imputation')
        
        plt.title(f'Distribution of {feature}')
        plt.legend()
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        
        plt.tight_layout()
        
        plot_path = os.path.join(output_folder, f"{feature}_distribution_comparison.png")
        plt.savefig(plot_path)
        plt.close()
    
    logging.info(f"Distribution plots saved in {output_folder}")

def check_extreme_values(input_df_before, input_df_after, threshold=5.0):
    """
    Check for extreme outliers introduced after imputation.
    """
    suspicious_features = []

    for feature in input_df_before.columns:
        before_max = input_df_before[feature].max()
        before_min = input_df_before[feature].min()
        after_max = input_df_after[feature].max()
        after_min = input_df_after[feature].min()

        if before_max != 0 and (after_max > threshold * before_max or after_max < before_max / threshold):
            suspicious_features.append((feature, 'max', before_max, after_max))
        if before_min != 0 and (after_min < before_min / threshold or after_min > threshold * before_min):
            suspicious_features.append((feature, 'min', before_min, after_min))

    suspicious_df = pd.DataFrame(
        suspicious_features, 
        columns=["Feature", "Type", "Before_Value", "After_Value"]
    )

    if not suspicious_df.empty:
        logging.warning(f"Found {len(suspicious_df)} suspicious extreme values after imputation!")

    return suspicious_df

In [ ]:
# Load imputed small data
data_imputed = pd.read_csv(f"CSV/exports/CRI-02/o1_impute_baselines/01_iterative/{dataset}_Iterative_{method_imputed}.csv")

# Plot distributions before vs after
plot_before_after_distributions(
    input_df_before=dataset,
    input_df_after=data_imputed,
    output_folder=f"figures/CRI-02/o1_impute_baselines/01_iterative/{method_imputed}/",
)

# Check for extreme values
extreme_values_df = check_extreme_values(
    input_df_before=dataset,
    input_df_after=data_imputed,
    threshold=5.0
)

# Save report if needed
if not extreme_values_df.empty:
    extreme_values_df.to_csv(f"CSV/exports/CRI-02/o1_impute_baselines/01_iterative/{dataset}_{method_imputed}_extreme_values.csv", index=False)
    logging.info("Extreme values report saved.")
else:
    logging.info("No suspicious extreme values detected.")

# Testing Field

In [ ]:
# Create a small subset of the dataframe (for faster testing)
small_data = o4_X_train.iloc[:450, :]  # pick first 50 features

In [ ]:
"""
Play with sample
"""

# Create a small subset of the dataframe (for faster testing)
small_data = o4_X_train.iloc[:450, :]  # pick first 50 features

# Test imputation on small_data
impute_with_iterative(
    input_df=small_data,
    output_path="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/small_o3_X_train_imputed_Iterative_ExtraTrees.csv",
    method="ExtraTrees",
    n_iter=20
)

In [ ]:
def plot_before_after_distributions(input_df_before, input_df_after, output_folder, sample_features=10):
    """
    Plot before and after distributions for random features and save the figures.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    features = list(input_df_before.columns)
    sampled_features = random.sample(features, min(sample_features, len(features)))
    
    logging.info(f"Plotting distributions for features: {sampled_features}")

    for feature in sampled_features:
        plt.figure(figsize=(10,5))
        
        plt.hist(input_df_before[feature].dropna(), bins=50, alpha=0.5, label='Before Imputation')
        plt.hist(input_df_after[feature].dropna(), bins=50, alpha=0.5, label='After Imputation')
        
        plt.title(f'Distribution of {feature}')
        plt.legend()
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        
        plt.tight_layout()
        
        plot_path = os.path.join(output_folder, f"{feature}_distribution_comparison.png")
        plt.savefig(plot_path)
        plt.close()
    
    logging.info(f"Distribution plots saved in {output_folder}")

def check_extreme_values(input_df_before, input_df_after, threshold=5.0):
    """
    Check for extreme outliers introduced after imputation.
    """
    suspicious_features = []

    for feature in input_df_before.columns:
        before_max = input_df_before[feature].max()
        before_min = input_df_before[feature].min()
        after_max = input_df_after[feature].max()
        after_min = input_df_after[feature].min()

        if before_max != 0 and (after_max > threshold * before_max or after_max < before_max / threshold):
            suspicious_features.append((feature, 'max', before_max, after_max))
        if before_min != 0 and (after_min < before_min / threshold or after_min > threshold * before_min):
            suspicious_features.append((feature, 'min', before_min, after_min))

    suspicious_df = pd.DataFrame(
        suspicious_features, 
        columns=["Feature", "Type", "Before_Value", "After_Value"]
    )

    if not suspicious_df.empty:
        logging.warning(f"Found {len(suspicious_df)} suspicious extreme values after imputation!")

    return suspicious_df

In [ ]:
# Load imputed small data
small_data_imputed = pd.read_csv("CSV/exports/CRI-02/o1_impute_baselines/01_iterative/small_o3_X_train_imputed_Iterative_ExtraTrees.csv")

# Plot distributions before vs after
plot_before_after_distributions(
    input_df_before=small_data,
    input_df_after=small_data_imputed,
    output_folder="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/plots_small_test/",
    #sample_features=10
)

# Check for extreme values
extreme_values_df = check_extreme_values(
    input_df_before=small_data,
    input_df_after=small_data_imputed,
    threshold=5.0
)

# Save report if needed
if not extreme_values_df.empty:
    extreme_values_df.to_csv("CSV/exports/CRI-02/o1_impute_baselines/01_iterative/extreme_values_report_small_test.csv", index=False)
    logging.info("Extreme values report saved.")
else:
    logging.info("No suspicious extreme values detected.")

In [ ]:
# ExtraTrees estimator
impute_with_iterative(
    input_df=o4_X_train,
    output_path="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o3_X_train_imputed_Iterative_ExtraTrees.csv",
    method="ExtraTrees",
    n_iter=20
)

In [ ]:
# BayesianRidge estimator
impute_with_iterative(
    input_df=o4_X_train,
    output_path="CSV/exports/CRI-02/o1_impute_baselines/01_iterative/o3_X_train_imputed_Iterative_BayesianRidge.csv",
    method="BayesianRidge",
    n_iter=20
)